## Seminar 1: Fun with Word Embeddings (3 points)

Today we gonna play with word embeddings: train our own little embedding, load one from   gensim model zoo and use it to visualize text corpora.

This whole thing is gonna happen on top of embedding dataset.

__Requirements:__  `pip install --upgrade nltk gensim bokeh` , but only if you're running locally.

In [1]:
# pip install --upgrade nltk gensim bokeh
# ! pip uninstall gensim=4.0.0
! pip install gensim==3.8.3

In [2]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

zsh:1: no matches found: https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1


In [3]:
import numpy as np

data = list(open("./quora.txt", encoding="utf-8"))
data[50]

"What TV shows or books help you read people's body language?\n"

In [4]:
data[0:10]

["Can I get back with my ex even though she is pregnant with another guy's baby?\n",
 'What are some ways to overcome a fast food addiction?\n',
 'Who were the great Chinese soldiers and leaders who fought in WW2?\n',
 'What are ZIP codes in the Bay Area?\n',
 'Why was George RR Martin critical of JK Rowling after losing the Hugo award?\n',
 'What can I do to improve my immune system?\n',
 'How is your relationship with your mother in law?\n',
 'How does one get Free PSN codes/Vita Codes?\n',
 'What is your review of osquery?\n',
 'How can I look smart and act smart?\n']

__Tokenization:__ a typical first step for an nlp task is to split raw data into words.
The text we're working with is in raw format: with all the punctuation and smiles attached to some words, so a simple str.split won't do.

Let's use __`nltk`__ - a library that handles many nlp tasks like tokenization, stemming or part-of-speech tagging.

In [21]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [22]:
# TASK: lowercase everything and extract tokens with tokenizer. 
# data_tok should be a list of lists of tokens for each line in data.

data_tok = [tokenizer.tokenize(d.lower()) for d in data]

In [23]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

In [24]:
print([' '.join(row) for row in data_tok[:2]])

["can i get back with my ex even though she is pregnant with another guy ' s baby ?", 'what are some ways to overcome a fast food addiction ?']


In [25]:
len(data_tok)

537272

__Word vectors:__ as the saying goes, there's more than one way to train word embeddings. There's Word2Vec and GloVe with different objective functions. Then there's fasttext that uses character-level models to train word embeddings. 

The choice is huge, so let's start someplace small: __gensim__ is another nlp library that features many vector-based models incuding word2vec.

In [11]:
from gensim.models import Word2Vec
model = Word2Vec(data_tok, 
                 size=32,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5).wv  # define context as a 5-word window around the target word

In [12]:
# now you can get word vectors !
model.get_vector('anything')

array([-1.3708725 ,  0.15484484, -0.11538529, -0.11822504, -1.688513  ,
        3.6897461 , -1.2680478 , -1.7015908 , -2.1592734 , -2.8217502 ,
       -1.3353859 , -0.17274253, -0.03597396,  2.3714035 ,  3.8228438 ,
        0.04153875,  2.2793868 , -1.9366903 ,  0.30141693,  2.755534  ,
       -3.5556018 , -2.5586743 ,  0.11142964,  2.696123  ,  1.8187184 ,
       -0.41993538,  2.822079  , -2.0559788 , -1.95803   , -2.7155693 ,
        0.19031744,  0.3994406 ], dtype=float32)

In [13]:
# or query similar words directly. Go play with it!
# model.most_similar('bread')

model.most_similar('pdf')

[('format', 0.9288365244865417),
 ('zip', 0.8488729000091553),
 ('xml', 0.831146240234375),
 ('audio', 0.8305941820144653),
 ('query', 0.8264321684837341),
 ('code', 0.8186598420143127),
 ('header', 0.8146935701370239),
 ('apk', 0.8094415664672852),
 ('mp3', 0.8013336658477783),
 ('json', 0.8000653982162476)]

In [14]:
a1 = model.get_vector("zip")
a2 = model.get_vector("pdf")
model.cosine_similarities(a1, model.vectors)

array([-0.0671482 ,  0.02694345,  0.09805726, ...,  0.4377001 ,
        0.22055517,  0.22504903], dtype=float32)

### Using pre-trained model

Took it a while, huh? Now imagine training life-sized (100~300D) word embeddings on gigabytes of text: wikipedia articles or twitter posts. 

Thankfully, nowadays you can get a pre-trained word embedding model in 2 lines of code (no sms required, promise).

In [5]:
import gensim.downloader as api
model = api.load('glove-twitter-100')

In [6]:
model.most_similar(positive=["coder", "money"], negative=["brain"])
# model.most_similar(positive=["remote"], negative=["hobby"])

[('broker', 0.5820155143737793),
 ('bonuses', 0.5424473285675049),
 ('banker', 0.5385112762451172),
 ('designer', 0.5197198390960693),
 ('merchandising', 0.4964233338832855),
 ('treet', 0.49220192432403564),
 ('shopper', 0.4920561909675598),
 ('part-time', 0.49128279089927673),
 ('freelance', 0.4843312203884125),
 ('aupair', 0.4796452522277832)]

In [7]:
words = sorted(model.vocab.keys(), 
               key=lambda word: model.vocab[word].count,
               reverse=True)[:1000]

print(words[::100])

['<user>', '_', 'please', 'apa', 'justin', 'text', 'hari', 'playing', 'once', 'sei']


### Visualizing word vectors

One way to see if our vectors are any good is to plot them. Thing is, those vectors are in 30D+ space and we humans are more used to 2-3D.

Luckily, we machine learners know about __dimensionality reduction__ methods.

Let's use that to plot 1000 most frequent words

In [8]:
words = sorted(model.vocab.keys(), 
               key=lambda word: model.vocab[word].count,
               reverse=True)[:1000]

print(words[::100])

['<user>', '_', 'please', 'apa', 'justin', 'text', 'hari', 'playing', 'once', 'sei']


In [9]:
# for each word, compute it's vector with model
word_vectors = []
for w in words:
    word_vectors.append(model.get_vector(w))
    
word_vectors = np.array(word_vectors)

In [10]:
type(word_vectors)


numpy.ndarray

In [11]:
assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), 100)
assert np.isfinite(word_vectors).all()

#### Linear projection: PCA

The simplest linear dimensionality reduction method is __P__rincipial __C__omponent __A__nalysis.

In geometric terms, PCA tries to find axes along which most of the variance occurs. The "natural" axes, if you wish.

<img src="https://github.com/yandexdataschool/Practical_RL/raw/master/yet_another_week/_resource/pca_fish.png" style="width:30%">


Under the hood, it attempts to decompose object-feature matrix $X$ into two smaller matrices: $W$ and $\hat W$ minimizing _mean squared error_:

$$\|(X W) \hat{W} - X\|^2_2 \to_{W, \hat{W}} \min$$
- $X \in \mathbb{R}^{n \times m}$ - object matrix (**centered**);
- $W \in \mathbb{R}^{m \times d}$ - matrix of direct transformation;
- $\hat{W} \in \mathbb{R}^{d \times m}$ - matrix of reverse transformation;
- $n$ samples, $m$ original dimensions and $d$ target dimensions;



In [12]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# map word vectors onto 2d plane with PCA. Use good old sklearn api (fit, transform)
# after that, normalize vectors to make sure they have zero mean and unit variance
scaller = StandardScaler()
pca = PCA(n_components=2)
word_vectors_pca = scaller.fit_transform(pca.fit_transform(word_vectors))

# and maybe MORE OF YOUR CODE here :)

In [13]:
assert word_vectors_pca.shape == (len(word_vectors), 2), "there must be a 2d vector for each word"
assert max(abs(word_vectors_pca.mean(0))) < 1e-5, "points must be zero-centered"
assert max(abs(1.0 - word_vectors_pca.std(0))) < 1e-2, "points must have unit variance"

#### Let's draw it!

In [14]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [15]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

# hover a mouse over there and see if you can identify the clusters

Figure(id='1004', ...)

### Visualizing neighbors with t-SNE
PCA is nice but it's strictly linear and thus only able to capture coarse high-level structure of the data.

If we instead want to focus on keeping neighboring points near, we could use TSNE, which is itself an embedding method. Here you can read __[more on TSNE](https://distill.pub/2016/misread-tsne/)__.

In [16]:
from sklearn.manifold import TSNE

# map word vectors onto 2d plane with TSNE. hint: don't panic it may take a minute or two to fit.
# normalize them as just lke with pca

tsne = TSNE(n_components=2)
word_tsne = scaller.fit_transform(tsne.fit_transform(word_vectors))

In [17]:
draw_vectors(word_tsne[:, 0], word_tsne[:, 1], color='green', token=words)

Figure(id='1114', ...)

### Visualizing phrases

Word embeddings can also be used to represent short phrases. The simplest way is to take __an average__ of vectors for all tokens in the phrase with some weights.

This trick is useful to identify what data are you working with: find if there are any outliers, clusters or other artefacts.

Let's try this new hammer on our data!


In [26]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    # 2. tokenize phrase
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros
    
    vector = np.zeros([model.vector_size], dtype='float32')
    
    w_vectors = []
    for w in tokenizer.tokenize(phrase.lower()):
        if w in model.vocab:
            w_vectors.append(model.get_vector(w))
    
    if len(w_vectors) > 0:
        vector = np.mean(w_vectors,0)

    # YOUR CODE
    
    return vector

In [27]:
get_phrase_embedding("I'm very sure. This never happened to me before...")

array([ 0.31807372,  0.25188074,  0.06748973,  0.07918166, -0.20551686,
        0.3099363 ,  0.12886   , -0.06323332,  0.1167265 ,  0.1529691 ,
       -0.02558171, -0.22829592, -4.4815    ,  0.02976158, -0.12603833,
       -0.1181161 , -0.23754632, -0.02389293, -0.47764084, -0.03727125,
        0.0933293 , -0.11675651, -0.07968461,  0.16103767,  0.22593975,
       -0.9288166 , -0.13147809, -0.27092585,  0.32907042, -0.12769467,
       -0.1002182 ,  0.05345418, -0.33383527,  0.16147001,  0.05949384,
        0.13145241,  0.04188244, -0.11879817,  0.09030958, -0.02781658,
       -1.0278689 ,  0.00457059,  0.21854158, -0.06651367,  0.23628668,
       -0.14505692,  0.17416383, -0.08423841, -0.12737857, -0.01618933,
       -0.16621883,  0.12946318,  0.17684542, -0.01754692,  0.12413007,
        0.01527582, -0.11908681,  0.11422   ,  0.15738223,  0.09719244,
        0.05083408,  0.0163896 ,  0.03656067, -0.11753249,  0.25182876,
       -0.10408742, -0.11623914, -0.11769509, -0.22145198,  0.07

In [28]:
vector = get_phrase_embedding("I'm very sure. This never happened to me before...")

assert np.allclose(vector[::10],
                   np.array([ 0.31807372, -0.02558171,  0.0933293 , -0.1002182 , -1.0278689 ,
                             -0.16621883,  0.05083408,  0.17989802,  1.3701859 ,  0.08655966],
                              dtype=np.float32))

In [29]:
# let's only consider ~5k phrases for a first run.
chosen_phrases = data[::len(data) // 1000]

# compute vectors for chosen phrases
phrase_vectors = np.array([get_phrase_embedding(ph) for ph in chosen_phrases])

In [30]:
assert isinstance(phrase_vectors, np.ndarray) and np.isfinite(phrase_vectors).all()
assert phrase_vectors.shape == (len(chosen_phrases), model.vector_size)

In [31]:
# map vectors into 2d space with pca, tsne or your other method of choice
# don't forget to normalize

phrase_vectors_2d = TSNE().fit_transform(phrase_vectors)

phrase_vectors_2d = (phrase_vectors_2d - phrase_vectors_2d.mean(axis=0)) / phrase_vectors_2d.std(axis=0)

In [32]:
draw_vectors(phrase_vectors_2d[:, 0], phrase_vectors_2d[:, 1],
             phrase=[phrase[:50] for phrase in chosen_phrases],
             radius=20,)

Figure(id='1235', ...)

Finally, let's build a simple "similar question" engine with phrase embeddings we've built.

In [33]:
# compute vector embedding for all lines in data
data_vectors = np.array([get_phrase_embedding(l) for l in data])

In [43]:
data_vectors.shape
data_vectors[1,:]

array([ 2.03387275e-01, -1.68813527e-01,  6.88512921e-02,  1.29120439e-01,
        4.75199968e-02,  2.32873917e-01,  7.69012719e-02, -8.24932754e-02,
        1.12726122e-01, -2.20229998e-01, -6.63480014e-02, -5.34394562e-01,
       -4.38643646e+00, -8.11745450e-02, -7.35079050e-02, -9.76834372e-02,
        1.96561813e-02, -2.91038513e-01, -1.56882182e-01, -1.17900103e-01,
        6.58281967e-02,  4.04846035e-02, -7.02763721e-02, -7.25217862e-03,
       -1.10284751e-02, -3.39105725e-01,  1.10118203e-02,  2.85152644e-01,
        2.10508093e-01,  4.39375378e-02, -1.60122454e-01,  3.50315869e-01,
       -2.13188455e-01,  1.66975074e-02,  8.23154673e-03, -6.15406372e-02,
        1.59391820e-01,  2.41981307e-03, -1.74478233e-01,  2.59105474e-01,
       -6.06980383e-01,  1.03271276e-01,  8.46343562e-02, -2.34344546e-02,
        3.75122070e-01,  1.02825634e-01, -1.16184451e-01,  3.05251896e-01,
       -6.25972915e-03, -1.35757908e-01, -7.54335299e-02, -6.74938262e-02,
       -7.47391731e-02,  

In [95]:
# import numpy.linalg.norm as norm

def find_nearest(query, k=10):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition, np.argsort
    """
    # YOUR CODE
    
    query_embedding = get_phrase_embedding(query)
    query_norm = np.linalg.norm(query_embedding);
    cosins = []
    for vec in data_vectors:
        vec_norm = np.linalg.norm(vec)  
        cosin = (query_embedding @ vec) / (vec_norm * query_norm)
        cosins.append(cosin)
    
#     args = np.argsort(cosins)[-k:]
#     print(f"{cosins[1]}")
    args = np.argsort(cosins)[::-1][2:k+2]
    print(f"{args}")
    
    return [data[i] for i in args]

#     return <YOUR CODE: top-k lines starting from most similar>

In [96]:
find_nearest(query="How do i enter the matrix?", k=10)

/var/folders/bj/_vsd3m557t1f_thrrlbwhjn80000gn/T/ipykernel_16872/4061656039.py:16: RuntimeWarning: invalid value encountered in float_scalars
  cosin = (query_embedding @ vec) / (vec_norm * query_norm)


[348630 194211 535949 248982 416639 358896 384831 499334 277540  78818]


['How do I get to the dark web?\n',
 'What should I do to enter hollywood?\n',
 'How do I use the Greenify app?\n',
 'What can I do to save the world?\n',
 'How do I win this?\n',
 'How do I think out of the box? How do I learn to think out of the box?\n',
 'How do I find the 5th dimension?\n',
 'How do I use the pad in MMA?\n',
 'How do I estimate the competition?\n',
 'What do I do to enter the line of event management?\n']

In [97]:
results = find_nearest(query="How do i enter the matrix?", k=10)

print(''.join(results))

assert len(results) == 10 and isinstance(results[0], str)
assert results[0] == 'How do I get to the dark web?\n'
assert results[3] == 'What can I do to save the world?\n'

/var/folders/bj/_vsd3m557t1f_thrrlbwhjn80000gn/T/ipykernel_16872/4061656039.py:16: RuntimeWarning: invalid value encountered in float_scalars
  cosin = (query_embedding @ vec) / (vec_norm * query_norm)


[348630 194211 535949 248982 416639 358896 384831 499334 277540  78818]
How do I get to the dark web?
What should I do to enter hollywood?
How do I use the Greenify app?
What can I do to save the world?
How do I win this?
How do I think out of the box? How do I learn to think out of the box?
How do I find the 5th dimension?
How do I use the pad in MMA?
How do I estimate the competition?
What do I do to enter the line of event management?



In [98]:
find_nearest(query="How does Trump?", k=10)

/var/folders/bj/_vsd3m557t1f_thrrlbwhjn80000gn/T/ipykernel_16872/4061656039.py:16: RuntimeWarning: invalid value encountered in float_scalars
  cosin = (query_embedding @ vec) / (vec_norm * query_norm)


[536175   9618 210436  37557  18780 187158 518962 203826 110684  45478]


['What does Donald Trump think about Israel?\n',
 'What books does Donald Trump like?\n',
 'What does Donald Trump think of India?\n',
 'What does India think of Donald Trump?\n',
 'What does Donald Trump think of China?\n',
 'What does Donald Trump think about Pakistan?\n',
 'What companies does Donald Trump own?\n',
 'What does Dushka Zapata think about Donald Trump?\n',
 'How does it feel to date Ivanka Trump?\n',
 'What does salesforce mean?\n']

In [99]:
find_nearest(query="Why don't i ask a question myself?", k=10)

/var/folders/bj/_vsd3m557t1f_thrrlbwhjn80000gn/T/ipykernel_16872/4061656039.py:16: RuntimeWarning: invalid value encountered in float_scalars
  cosin = (query_embedding @ vec) / (vec_norm * query_norm)


[481169 289957 421503 323721 374877 468871 153603 478070  10739 194652]


["Why don't I get a date?\n",
 "Why do you always answer a question with a question? I don't, or do I?\n",
 "Why can't I ask a question anonymously?\n",
 "Why don't I get a girlfriend?\n",
 "Why don't I have a boyfriend?\n",
 "I don't have no question?\n",
 "Why can't I take a joke?\n",
 "Why don't I ever get a girl?\n",
 "Can I ask a girl out that I don't know?\n",
 "Why don't I have a girlfriend?\n"]

__Now what?__
* Try running TSNE on all data, not just 1000 phrases
* See what other embeddings are there in the model zoo: `gensim.downloader.info()`
* Take a look at [FastText](https://github.com/facebookresearch/fastText) embeddings
* Optimize find_nearest with locality-sensitive hashing: use [nearpy](https://github.com/pixelogik/NearPy) or `sklearn.neighbors`.